In [1]:
import os
import h5py as h5
import pandas as pd
import geopandas as gpd
import numpy as np
from probabalistic_displacement_scripts import get_exceedance_bar_chart_data

run_name = 'sz_fq_wellington_1km'

prob_thresholds = [0.1, 0.02]
exceed_type_list = ['up', 'down']

weighted = True

results_dir = os.path.join('..', 'results')

if weighted:
    h5_file = os.path.join(results_dir, run_name, 'weighted_mean_PPE_dict_uniform.h5')

PPEh5 = h5.File(h5_file, 'r')

sites = [*PPEh5.keys()]
metadata_keys = ['branch_weights', 'branch_ids', 'thresholds', 'threshold_vals', 'sigma_lims']
for meta in metadata_keys:
    if meta in sites:
        sites.remove(meta)

sites = [sites[ix] for ix in np.argsort(np.array(sites).astype(int))]

out_columns = ['Site ID', 'Lon' ,'Lat']
for exceed_type in exceed_type_list:
    for prob in prob_thresholds:
        out_columns.append(f'{prob*100}% {exceed_type} POE (m)')
        out_columns.append(f'{prob*100}% {exceed_type} sigma (m)')

output = pd.DataFrame(columns=out_columns)


In [2]:
lons = []
lats = []

for siteID in sites:
    site = PPEh5[siteID]
    lon, lat, _ = site['site_coords'][()]
    lons.append(lon)
    lats.append(lat)

locs = gpd.GeoDataFrame(geometry=gpd.points_from_xy(lons, lats)).set_crs(epsg=2193)
locs.geometry = locs.geometry.to_crs(epsg=4326)

csv_dict = {'Site ID': sites, 'Lon': locs.geometry.x.map(lambda x: round(x, 6)), 'Lat': locs.geometry.y.map(lambda x: round(x, 6))}

for exceed_type in exceed_type_list:
    for ii, prob in enumerate(prob_thresholds):
        disps, errs = get_exceedance_bar_chart_data(site_PPE_dictionary=PPEh5, exceed_type=exceed_type,
                                                    site_list=sites, probability=prob, weighted=weighted, err_index=[1,2])
        csv_dict[f'{prob*100}% {exceed_type} POE (m)'] = [f"{disp:.02f}" for disp in disps]
        csv_dict[f'{prob*100}% {exceed_type} +1 sigma (m)'] = [f"{max(err):.02f}" for err in errs]
        csv_dict[f'{prob*100}% {exceed_type} -1 sigma (m)'] = [f"{min(err):.02f}" for err in errs]


out_df = pd.DataFrame(csv_dict)
out_df.to_csv(os.path.join(results_dir, run_name, 'exceedance_bar_chart_data.csv'), index=False)

PermissionError: [Errno 13] Permission denied: '..\\results\\sz_fq_wellington_1km\\exceedance_bar_chart_data.csv'